In [ ]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
import numpy as np

## Vieweing NLCD

In [ ]:
M.set_center(-120.32, 47.84, 7)

In [ ]:
sc = SparkContext(appName="NLCD Viewer")
geopysc = GeoPyContext(sc)

In [ ]:
catalog_uri = "s3://azavea-datahub/catalog"
layer_name = "nlcd-tms-epsg3857"

In [ ]:
data = GeoTrellisCatalogLayerData(geopysc, 
                                  catalog_uri, 
                                  layer_name,
                                  SPATIAL)

In [ ]:
from geonotebook.vis.geotrellis.render_methods import render_nlcd

M.add_layer(data, render_tile=render_nlcd)

## Viewing reclassified tiles

In [ ]:
def reclass(tile):
    # Planted/Cultivated
    # See https://www.mrlc.gov/nlcd11_leg.php
    result = tile.copy()
    result[np.ma.where((80 <= tile) & (tile < 90))] = 1
    result[np.ma.where((tile  < 80) | (90 <= tile))] = 0
    return result
     
cmap = { 0 : "#00000000", 1: "#CA9146FF" }
cmap_render = single_band_render_from_color_map(cmap)

def reclass_render(tile):
    reclassed = reclass(tile[0])
    return cmap_render(reclassed)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(data, render_tile=reclass_render)

## Chattanooga geometry

In [ ]:
!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

In [ ]:
from functools import partial
import fiona
import json
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_poly = transform(project, geom)
chatta_poly

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
from geonotebook.wrappers import VectorData
vd = VectorData("/tmp/mask.json")
name = "Outline"
M.add_layer(vd, name=name)

In [ ]:
M.set_center(center.x, center.y, 9)

## Fetching an RDD of NLCD masked to Chattanooga

In [ ]:
MAX_ZOOM = 12
query_rdd = query(geopysc, SPATIAL, catalog_uri, layer_name, 12, intersects=chatta_poly)
converted_rdd = query_rdd.convert_data_type("int8")

In [ ]:
masked = converted_rdd.mask([chatta_poly])
rd = RddRasterData(masked)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(rd, render_tile=render_nlcd)

## Reclassifying an RDD

In [ ]:
nprdd = converted_rdd.to_numpy_rdd()

In [ ]:
def mapper(tile):
    arr = tile['data'][0]
    np.ma.masked_where(arr == tile['no_data_value'], arr)
    tile['data'] = np.array([reclass(arr)])
    return tile
mapped = nprdd.mapValues(mapper).cache()
gtRdd = TiledRasterRDD.from_numpy_rdd(geopysc, SPATIAL, mapped, query_rdd.layer_metadata)
reclassed = gtRdd.mask([chatta_poly])

In [ ]:
cmap = { 0 : "#00FFAA88", 1: "#CA9146FF" }
cmap_render = single_band_render_from_color_map(cmap)
def render_tile(tile):
    arr = tile[0]
    return cmap_render(arr)

In [ ]:
rd = RddRasterData(reclassed)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(rd, render_tile=render_tile)

## Saving the reclassified layer locally

In [ ]:
local_catalog_uri = "file://catalog"

In [ ]:
# Reproject to bring the tile sizes to 256
retiled = reclassed.reproject("EPSG:3857", scheme=ZOOM)

In [ ]:
for layer_rdd in retiled.pyramid(retiled.zoom_level, 0):
    write(local_catalog_uri, "cultivated-land-cover", layer_rdd)


## Viewing the local layer

In [ ]:
data = GeoTrellisCatalogLayerData(geopysc, 
                                  local_catalog_uri, 
                                  "cultivated-land-cover",
                                  SPATIAL)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(data, render_tile=render_tile)